In [1]:
from pathlib import Path
import pandas as pd

In [2]:
LOAD_LOAD_DIR = Path(
    Path.cwd().parent, "data.nosync", "transformed_data", "load_transformed.parquet"
)
WEATHER_LOAD_DIR = Path(
    Path.cwd().parent, "data.nosync", "transformed_data", "weather_transformed.parquet"
)
MERGED_SAVE_DIR = Path(
    Path.cwd().parent, "data.nosync", "transformed_data", "merged.parquet"
)

In [3]:
weather_df = pd.read_parquet(WEATHER_LOAD_DIR)
load_df = pd.read_parquet(LOAD_LOAD_DIR)
load_df['Date'] = load_df['Time Stamp'].dt.date
load_df['Date'] = pd.to_datetime(load_df['Date'])

In [4]:
weather_df.head()

,Forecast Date,Vintage Date,Vintage,Station ID,Max Temp,Min Temp,Max Wet Bulb,Min Wet Bulb,lat,lon,grid_zone,Forecast Year,Forecast Month,Forecast Day,Vintage Year,Vintage Month,Vintage Day
0,2011-03-02,2011-02-27,Actual,ALB,2.777778,-5.555556,0.000000,-6.111111,42.652580,-73.756233,CAPITL,2011,3,2,2011,2,27
1,2011-03-02,2011-02-27,Actual,ART,3.888889,-7.222222,1.666667,-7.777778,43.974785,-75.910759,MHK VL,2011,3,2,2011,2,27
2,2011-03-02,2011-02-27,Actual,BGM,1.111111,-3.333333,-0.555556,-3.888889,42.098843,-75.920647,CENTRL,2011,3,2,2011,2,27
3,2011-03-02,2011-02-27,Actual,BUF,3.333333,-3.333333,1.666667,-3.333333,42.880230,-78.878738,WEST,2011,3,2,2011,2,27
4,2011-03-02,2011-02-27,Actual,ELM,3.888889,-2.777778,1.666667,-3.333333,42.084972,-76.798553,CENTRL,2011,3,2,2011,2,27


In [5]:
load_df.head()

,Time Stamp,Time Zone,Name,PTID,Load,Year,Month,Day,Minute,Hour,Date
0,2016-06-22,EDT,CAPITL,61757.0,1252.099976,2016,6,22,0,0,2016-06-22
1,2016-06-22,EDT,CENTRL,61754.0,1594.699951,2016,6,22,0,0,2016-06-22
2,2016-06-22,EDT,DUNWOD,61760.0,717.099976,2016,6,22,0,0,2016-06-22
3,2016-06-22,EDT,GENESE,61753.0,1028.699951,2016,6,22,0,0,2016-06-22
4,2016-06-22,EDT,HUD VL,61758.0,1065.500000,2016,6,22,0,0,2016-06-22


In [6]:
# As we have multiple values for forecasts for each date and name (driven by different vintage dates), we will take the average to create a single forcast for each date and region
weather_grouped = weather_df.groupby(['Forecast Date', 'grid_zone'], as_index = False).mean(numeric_only = True)[['Forecast Date', 'grid_zone', 'Max Temp', 'Min Temp', 'Max Wet Bulb', 'Min Wet Bulb']]
weather_grouped.head()

,Forecast Date,grid_zone,Max Temp,Min Temp,Max Wet Bulb,Min Wet Bulb
0,2008-09-08,CAPITL,26.333333,17.000000,20.666667,15.222222
1,2008-09-08,CENTRL,24.592593,14.629630,18.481481,13.666667
2,2008-09-08,DUNWOD,26.444444,18.777778,20.666667,17.111111
3,2008-09-08,GENESE,23.444444,15.444444,17.777778,13.666667
4,2008-09-08,HUD VL,26.666667,16.888889,20.777778,15.833333


In [7]:
merged_df = load_df.merge(weather_grouped, left_on = ['Date', 'Name'], right_on=['Forecast Date', 'grid_zone'])

In [8]:
merged_df.shape

(15509980, 17)

In [9]:
merged_df.head()

,Time Stamp,Time Zone,Name,PTID,Load,Year,Month,Day,Minute,Hour,Date,Forecast Date,grid_zone,Max Temp,Min Temp,Max Wet Bulb,Min Wet Bulb
0,2016-06-22 00:00:00,EDT,CAPITL,61757.0,1252.099976,2016,6,22,0,0,2016-06-22,2016-06-22,CAPITL,27.777778,15.0,18.444444,11.777778
1,2016-06-22 00:05:00,EDT,CAPITL,61757.0,1239.199951,2016,6,22,5,0,2016-06-22,2016-06-22,CAPITL,27.777778,15.0,18.444444,11.777778
2,2016-06-22 00:10:00,EDT,CAPITL,61757.0,1241.900024,2016,6,22,10,0,2016-06-22,2016-06-22,CAPITL,27.777778,15.0,18.444444,11.777778
3,2016-06-22 00:15:00,EDT,CAPITL,61757.0,1240.900024,2016,6,22,15,0,2016-06-22,2016-06-22,CAPITL,27.777778,15.0,18.444444,11.777778
4,2016-06-22 00:20:00,EDT,CAPITL,61757.0,1226.900024,2016,6,22,20,0,2016-06-22,2016-06-22,CAPITL,27.777778,15.0,18.444444,11.777778


In [10]:
merged_df = merged_df.drop(['Date', 'Forecast Date', 'grid_zone', 'Time Zone'], axis = 1) # Dropping duplicated and unused columns creating through merge

In [ ]:
merged_df.to_parquet(MERGED_SAVE_DIR)